In [1]:
import numpy as np
import pythonToC
import parsing_yacc 
from parsing_lex import MyLexer
import pythonTopython2
import time

def measure_execution_time(input_code, mode):
    start_time = time.time()
    output = deploy_parser(input_code, mode)
    execution_time = time.time() - start_time
    return output, execution_time

def deploy_parser(input, mode): 
    output = ""

    for line in input.split("\n"):
        if line == "":
            continue
        result = parsing_yacc.main(line, mode)
        if isinstance(result, str):
            output += result + "\n"

    return output

def run_trials(trials, array_lengths):
    results = []
    for trial in range(trials):
        for array_length in array_lengths:
            input_code = f"""
a = np.array({{np.random.randint(0, 100, {array_length})}})
b = np.array({{np.random.randint(0, 100, {array_length})}})

c = np.add(a, b)
d = np.subtract(a, b)
e = np.sum(a)
f = np.dot(a, b)
"""
            c_output, c_time = measure_execution_time(input_code, "C")
            python_output, python_time = measure_execution_time(input_code, "Python")
            results.append((trial, array_length, c_time, python_time))
    return results

# Create an instance of MyLexer
lexer = MyLexer()

# Build the lexer
lexer.build()

# Test the lexer on the input code
input_code = """
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

c = np.add(a, b)
d = np.subtract(a, b)
e = np.sum(a)
f = np.dot(a, b)
"""
lexer.test(input_code)

trials = 3
array_lengths = [10, 100, 1000]

results = run_trials(trials, array_lengths)

# Print the results
for result in results:
    print("Trial:", result[0])
    print("Array Length:", result[1])
    print("C Code Generation Time:", result[2])
    print("Python Code Generation Time:", result[3])
    print()


LexToken(VARIABLE,'a',1,1)
LexToken(EQUALS,'=',1,3)
LexToken(ARRAY,'np.array',1,5)
LexToken(LPAREN,'(',1,13)
LexToken(LIST,'[1, 2, 3]',1,14)
LexToken(RPAREN,')',1,23)
LexToken(VARIABLE,'b',1,25)
LexToken(EQUALS,'=',1,27)
LexToken(ARRAY,'np.array',1,29)
LexToken(LPAREN,'(',1,37)
LexToken(LIST,'[4, 5, 6]',1,38)
LexToken(RPAREN,')',1,47)
LexToken(VARIABLE,'c',1,50)
LexToken(EQUALS,'=',1,52)
LexToken(NP_ADD,'np.add',1,54)
LexToken(LPAREN,'(',1,60)
LexToken(VARIABLE,'a',1,61)
LexToken(COMMA,',',1,62)
LexToken(VARIABLE,'b',1,64)
LexToken(RPAREN,')',1,65)
LexToken(VARIABLE,'d',1,67)
LexToken(EQUALS,'=',1,69)
LexToken(NP_SUBTRACT,'np.subtract',1,71)
LexToken(LPAREN,'(',1,82)
LexToken(VARIABLE,'a',1,83)
LexToken(COMMA,',',1,84)
LexToken(VARIABLE,'b',1,86)
LexToken(RPAREN,')',1,87)
LexToken(VARIABLE,'e',1,89)
LexToken(EQUALS,'=',1,91)
LexToken(NP_SUM,'np.sum',1,93)
LexToken(LPAREN,'(',1,99)
LexToken(VARIABLE,'a',1,100)
LexToken(RPAREN,')',1,101)
LexToken(VARIABLE,'f',1,103)
LexToken(EQUALS,'=',1